In [30]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from glob import glob

In [85]:
def lss(expr = '*.*'):
    return glob(expr)

list_files = lss('*.xlsx')

In [86]:
years_files=["2018","2019","2020"]#reemplazar esto por regex que permita sacar los años de los archivos en el dir
df_dict={}
for i in years_files:     
    df_dict[i]=f"df_{i}"

In [87]:
for i in range(len(list_files)):
    df_dict[years_files[i]]= pd.read_excel(list_files[i], index_col=0) 

In [88]:
#Transformations to remove the header from the Agencia Nacional de Hidrocarburos.  
for i in years_files: 
    df_dict[i]=df_dict[i].dropna(thresh=1) #delete rows with just nan values
    df_dict[i]=df_dict[i].reset_index() #add dpto back as a col
    df_dict[i].columns=df_dict[i].iloc[0].str.lower() #set new first row as cols names
    df_dict[i]=df_dict[i].drop(df_dict[i].index[0]) #drop row in the df with cols names
    df_dict[i]= df_dict[i].drop(df_dict[i][df_dict[i]["campo"].isnull() & df_dict[i]["operadora"].isnull() 
                                           & df_dict[i]["municipio"].isnull()].index)#Eliminate the rows where the value of 
    #the field, operator and municipality is null because it  would not be possible to determine these data with the 
    #remaining information. 


In [89]:
January= df_dict['2020'].columns.get_loc("enero")
df_dict['2020']['Total_Prod']=df_dict['2020'].iloc[:,January::].fillna(0).sum(axis=1)

In [90]:
df_dict['2020'].isnull().sum()

0
departamento    0
municipio       0
operadora       0
contrato        0
campo           0
enero           0
febrero         0
marzo           0
abril           0
mayo            0
junio           0
julio           0
agosto          0
Total_Prod      0
dtype: int64

In [91]:
#Eliminar duplicados, si los hay
df_dict['2020']=df_dict['2020'].drop_duplicates()

In [92]:
df_dict['2020']

,departamento,municipio,operadora,contrato,campo,enero,febrero,marzo,abril,mayo,junio,julio,agosto,Total_Prod
1,ANTIOQUIA,PUERTO NARE,ECOPETROL S.A.,TECA COCORNA,AREA TECA-COCORNA,1765.84,1689.32,1826.93,1537.57,1238.72,1049.21,975.58,949,11032.17
2,ANTIOQUIA,PUERTO TRIUNFO,ECOPETROL S.A.,TECA COCORNA,AREA TECA-COCORNA,119.7,118.32,81.96,0,0,0,14.05,104.08,438.11
3,ANTIOQUIA,YONDO,ECOPETROL S.A.,MAGDALENA MEDIO,CASABE,11636.6,11584.2,11488.2,10711.5,9483.4,9402.73,9599.38,9527.73,83433.72
4,ANTIOQUIA,YONDO,ECOPETROL S.A.,MAGDALENA MEDIO,CASABE SUR,1852.6,1763.47,1831.5,1838.38,1539.16,1541.67,1573.16,1514.94,13454.88
5,ANTIOQUIA,PUERTO NARE,MANSAROVAR ENERGY COLOMBIA LTD,NARE,NARE SUR,187.33,184.97,173.58,191.32,193.94,166.74,168.84,152.02,1418.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,TOLIMA,CHAPARRAL,HOCOL S.A.,TOLIMA,RÍO SALDAÑA,857.97,816.66,1282.97,1287.09,1214.32,1185.98,1167.73,1133.84,8946.56
410,TOLIMA,ORTEGA,HOCOL S.A.,TOLDADO,TOLDADO,259.71,282.3,312.79,319.39,353.9,435.58,365.26,364.62,2693.55
411,TOLIMA,PIEDRAS,HOCOL S.A.,PULI,TOQUI TOQUI,356.76,374.42,463.51,404.32,365.96,358.03,348.97,362.87,3034.84
412,TOLIMA,ALVARADO,HOCOL S.A.,ARMERO,TOTARE,72.13,73.19,73,70.43,29.06,0,0,0,317.81


In [93]:
Highest_Prod=df_dict['2020'].groupby('campo')['Total_Prod'].sum().reset_index(name ='Anual_Prod')
#Seleccionar los campos con mayor producción 
Highest_Prod.sort_values(by=['Anual_Prod'], ascending=False).iloc[0:5]

,campo,Anual_Prod
313,RUBIALES,857001.07
72,CASTILLA,537068.14
92,CHICHIMENE,388023.81
74,CASTILLA NORTE,380005.77
298,QUIFA,304783.67


In [99]:
Op_Field_2018=df_dict['2018'].groupby(['operadora','departamento'])['campo'].count().reset_index(name ='#Prod_Fields')
display(Op_Field_2018[(Op_Field_2018['departamento']=='CASANARE')&(Op_Field_2018['#Prod_Fields']>5)].count())
display(Op_Field_2018[(Op_Field_2018['departamento']=='CASANARE')&(Op_Field_2018['#Prod_Fields']>5)])

operadora       8
departamento    8
#Prod_Fields    8
dtype: int64

,operadora,departamento,#Prod_Fields
5,CEPSA COLOMBIA S.A.,CASANARE,6
8,COLOMBIA ENERGY DEVELOPMENT CO,CASANARE,6
16,ECOPETROL S.A.,CASANARE,10
32,Frontera Energy Colombia Corp Sucursal Colombia,CASANARE,37
38,GEOPARK COLOMBIA S.A.S.,CASANARE,22
67,NEW GRANADA ENERGY CORPORATION SUCURSAL COLOMBIA,CASANARE,6
75,PAREX RESOURCES COLOMBIA LTD. SUCURSAL,CASANARE,7
76,PERENCO COLOMBIA LIMITED,CASANARE,36


In [97]:
Jan_2018= df_dict['2018'].columns.get_loc("enero")
df_dict['2018']['Total_Prod']=df_dict['2018'].iloc[:,Jan_2018::].fillna(0).sum(axis=1)
Cont_Highest_Prod= df_dict['2018'].groupby('contrato')['Total_Prod'].sum().reset_index(name ='Anual_Prod')
Cont_Highest_Prod.sort_values(by=['Anual_Prod'], ascending=False).iloc[0:5]

,contrato,Anual_Prod
43,CUBARRAL,4.354378e+06
138,RUBIALES,2.867224e+06
91,LLA 34,1.451159e+06
130,QUIFA,1.108877e+06
75,LA CIRA INFANTAS,1.084830e+06


In [100]:
Op_High_Prod_2019= df_dict['2019'].groupby('operadora')['agosto'].sum().reset_index(name ='Prod_Aug')
Op_High_Prod_2019.sort_values(by=['Prod_Aug'], ascending=False).iloc[0:10]

,operadora,Prod_Aug
7,ECOPETROL S.A.,473986.41
10,Frontera Energy Colombia Corp Sucursal Colombia,81441.68
11,GEOPARK COLOMBIA S.A.S.,69154.42
24,OCCIDENTAL DE COLOMBIA LLC,53299.12
9,EQUION ENERGÍA LIMITED,34950.01
13,GRAN TIERRA ENERGY COLOMBIA LTD,29555.20
21,MANSAROVAR ENERGY COLOMBIA LTD,25256.75
14,HOCOL S.A.,19614.02
28,PAREX RESOURCES COLOMBIA LTD. SUCURSAL,15727.97
3,CEPSA COLOMBIA S.A.,12702.09
